In [1]:
import torch
from pipeline_stable_diffusion_3 import StableDiffusion3Pipeline
from transformer_sd3 import SD3Transformer2DModel
from module import Module
transformer = SD3Transformer2DModel.from_pretrained("stabilityai/stable-diffusion-3.5-large", torch_dtype=torch.bfloat16, subfolder="transformer")
pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-large", torch_dtype=torch.bfloat16)
pipe.transformer = transformer

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
pipe = pipe.to("cuda")
prompt = "A cup with a dog photo on it"
negative_prompt="sky"
module = Module().cuda()
image1 = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=28,
    guidance_scale=4,
    module=module
).images[0] 

  0%|          | 0/28 [00:00<?, ?it/s]

In [3]:
import copy
intermediate_latents1 = copy.deepcopy(pipe.intermediate_latents)
intermediate_prompt_embeds1 = copy.deepcopy(pipe.intermediate_prompt_embeds)
pred1 = copy.deepcopy(pipe.pred)

In [4]:
pipe = pipe.to("cuda")
prompt = "A cup with a dog photo on it"
negative_prompt="sky"
image2 = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=28,
    guidance_scale=4,
    module=module
).images[0] 

  0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
from judge import ask_gpt 
import numpy as np
from PIL import Image


ask_gpt(image1, image2, prompt, negative_prompt) 

[20.0, 13.0]

In [6]:
intermediate_latents2 = copy.deepcopy(pipe.intermediate_latents)
intermediate_prompt_embeds2 = copy.deepcopy(pipe.intermediate_prompt_embeds)
pred2 = copy.deepcopy(pipe.pred)

In [7]:
pipe.pred[-1].shape

torch.Size([1, 32, 32])

In [ ]:
def compute_term(module, intermediate_latents, intermediate_prompt_embeds, pred):
    term = 0
    
    for i in range(len(intermediate_latents)):
        latents = intermediate_latents[i].cuda().float().unsqueeze(0)
        prompt_embeds = intermediate_prompt_embeds[i].cuda().float().unsqueeze(0)
        logistic = module(latents, prompt_embeds)
        prob = torch.softmax(logistic, dim=1) 
        # print(prob[0,:,10,10].argmax().item(), pred[i][:,10,10].item(), prob[0,:,10,10].max().item())
        prob = prob.reshape(prob.shape[1], -1) 
        pred_ = pred[i].reshape(-1)
        # print(prob.shape, pred_.shape)
        # selected_prob = prob[pred_] 
        selected_prob = torch.gather(prob, 0, pred_.unsqueeze(0).long()).squeeze(0)
        selected_log_prob = torch.log(selected_prob + 1e-10)
        term += torch.mean(selected_log_prob).item()
        # print(torch.mean(selected_log_prob).item())
    print(f"Term: {term/len(intermediate_prompt_embeds)}")
    print(f"{np.exp(term/len(intermediate_prompt_embeds))}")
compute_term(module, intermediate_latents1, intermediate_prompt_embeds1, pred1)

torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size([1024])
torch.Size([20, 1024]) torch.Size(